In [2]:
import xarray as xr
from config import POP_DATA_SRC

In [3]:
population_file = 'population_count_2000-2020.nc'

In [16]:
population_file = 'population_density_2000-2020.nc'

In [17]:
pop_file = POP_DATA_SRC / population_file
pop = xr.open_dataset(str(pop_file))
pop['time'] = pop['time.year']
pop = pop.rename({'time': 'year'})
pop.population.to_netcdf(str(pop_file) + '_new')